In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from DecisionTree_recursive import DecisionTree

In [12]:
%%time
from DecisionTree_recursive import DecisionTree
dt = DecisionTree(max_depth = 30, min_node = 20, min_leaf=10, threshold = 1e-3)
tree = dt.build_tree(train_data, 20, dt.get_gini(train_data))
dt.predict(train_data, tree)

Wall time: 659 ms


In [11]:
np.mean(train_data.predict == train_data.label)

0.10066643283058575

## Read

In [4]:
## read train data
data = pd.read_table('training.txt', sep=' ', header=None)
# name label
data.rename(columns={0:'label'}, inplace=True)
# sepearte labels
label = data.iloc[:, 0]

# remove index: from data
data = data.iloc[:, 1:].applymap(lambda x: x[x.find(':')+1:])

# rename columns and merge label
data.columns = list(range(data.shape[1]))
data = data.join(label)
# label: int, other columns: str

In [5]:
## read test data
test = pd.read_table('testing.txt', sep=' ', header=None)

# remove index: from data
test = test.applymap(lambda x: x[x.find(':')+1:])

In [6]:
# train test split
n = data.shape[0]
idx = np.random.randint(0, n, int(n/20))
val_data = data.iloc[idx,:]
train_data = data.drop(idx)

# data indexing
train_data.index = range(0, train_data.shape[0])
val_data.index = range(0, val_data.shape[0])

## Tuning

In [15]:
%%time
parameters = {'max_depth':[10, 15, 20, 25, 30], 'min_samples_split': [2, 3, 5, 10, 20],
              'min_samples_leaf':[2, 3, 5, 10], 'min_impurity_decrease':[1e-2, 3e-3, 1e-3, 3e-4]}
dt = DecisionTreeClassifier()
grid_dt = GridSearchCV(dt, parameters, cv=5)
grid_dt.fit(train_data.iloc[:,:-1], train_data.label)

print(grid_dt.best_estimator_)
print(grid_dt.best_score_)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.01, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
1.0
Wall time: 4min 46s


In [16]:
dt = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            min_impurity_decrease=1e-4, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=30,
            min_weight_fraction_leaf=0.0, splitter='random')
dt.fit(train_data.iloc[:,:-1], train_data.label)
pred = dt.predict(train_data.iloc[:,:-1])
print(np.mean(pred ==train_data.label))
pred = dt.predict(val_data.iloc[:,:-1])
print(np.mean(pred==val_data.label))

with open("tree.txt", "w") as f:
    f = tree.export_graphviz(dt, out_file=f)

1.0


ValueError: Number of features of the model must match the input. Model n_features is 129 and input n_features is 128 

## Save

In [23]:
with open('result.txt', 'w') as fp:
    [fp.write(str(predict)+'\n') for predict in pred]